In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import rnn_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'rnn'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 15
EXECUTIONS_PER_TRIAL = 2

TOP_N = 4
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_rnn_type = hp.Choice("rnn_type", values=["lstm", "gru"])
    hp_n_neurons = hp.Choice("n_neurons", values=[16, 32, 64, 128, 256, 512])
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=0, max_value=1, default=0)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization")

    model = bare_model_builder(hp_rnn_type, hp_n_neurons, hp_n_hidden_layers, hp_input_batch_normalization)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 lstm (LSTM)                 (None, 16)                1152      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 15 Complete [00h 01m 15s]
val_loss: 8.675355434417725

Best val_loss So Far: 8.635008811950684
Total elapsed time: 00h 17m 53s


# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'rnn_type': 'gru', 'n_neurons': 128, 'n_hidden_layers': 0, 'input_batch_normalization': True}
Number of parameters: 50435
========== Model 1 ==========
{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 1, 'input_batch_normalization': True}
Number of parameters: 37891
========== Model 2 ==========
{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 0, 'input_batch_normalization': True}
Number of parameters: 12931
========== Model 3 ==========
{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 1, 'input_batch_normalization': True}
Number of parameters: 37891


# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'rnn_type': 'gru', 'n_neurons': 128, 'n_hidden_layers': 0, 'input_batch_normalization': True}
Number of parameters: 50435
Got score: 8.8237 (8.8646, 8.7828)
Got score: 8.3710 (8.3631, 8.3790)
Got score: 8.1263 (8.1510, 8.1016)
Got score: 8.4575 (8.4820, 8.4331)
Got score: 8.7729 (8.8165, 8.7294)


{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 1, 'input_batch_normalization': True}
Number of parameters: 37891
Got score: 8.8606 (8.9856, 8.7356)
Got score: 8.5523 (8.5471, 8.5575)
Got score: 8.3271 (8.5486, 8.1055)
Got score: 8.5865 (8.6245, 8.5485)
Got score: 8.7567 (8.7735, 8.7398)


{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 0, 'input_batch_normalization': True}
Number of parameters: 12931
Got score: 8.9018 (9.0328, 8.7708)
Got score: 8.4277 (8.4960, 8.3593)
Got score: 8.1880 (8.1942, 8.1819)
Got score: 8.4861 (8.4690, 8.5032)
Got score: 8.7689 (8.7833, 8.7546)


{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 1, 'input_batch_normalization': True}
Number of parameters: 37891
Got score: 8.7644 (8.8080, 8.7209)
Got score: 8.4057 (8.3664, 8.4449)
Got score: 8.2076 (8.1306, 8.2845)
Got score: 8.6421 (8.7036, 8.5805)
Got score: 8.9165 (9.0850, 8.7479)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.51,0.26,50435
1,8.62,0.18,37891
2,8.55,0.25,12931
3,8.59,0.25,37891
